In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import branca.colormap as cmp
from branca.element import MacroElement
from jinja2 import Template

import osmnx as ox
import networkx as nx
import folium

In [12]:
google_elevation_api_key = r''

# SI 507 Final Project


I-80 Sacramento, CA to Soda Springs, CA:
- Google Maps Route:
    - https://goo.gl/maps/z4iBs7TgRo5Tn24B8
- Start Address:
    - 3428 Auburn Blvd, Sacramento, CA 95821
- End Address:
    - 21784 Donner Pass Rd, Soda Springs, CA 95728

In [3]:
def get_route_gdfs(graph, route):
    """
    Extracts node and edge geodataframes for a specific route from an OSMnx graph.

    Parameters:
    graph (networkx.MultiDiGraph): The input graph representing the road network.
    route (list): A list of node IDs in the order they appear in the route.

    Returns:
    tuple: A tuple containing two GeoDataFrames:
        - route_nodes_gdf (geopandas.GeoDataFrame): A GeoDataFrame with
        the geometries and attributes of the nodes in the route.
        - route_edges_gdf (geopandas.GeoDataFrame): A GeoDataFrame with
        the geometries and attributes of the edges in the route, excluding
        some columns ('oneway', 'lanes', 'bridge', 'reversed', 'geometry').
    """
    # Convert the graph to GeoDataFrames for nodes and edges
    nodes_gdfs, edges_gdfs = ox.graph_to_gdfs(graph)
    edges_gdfs = edges_gdfs.reset_index()

    # Filter the nodes GeoDataFrame to include only the nodes in the route
    route_nodes_gdf = nodes_gdfs[nodes_gdfs.index.isin(route)]

    # Filter the edges GeoDataFrame to include only the edges in the route
    route_edges_gdf = edges_gdfs[edges_gdfs['u'].isin(route) & edges_gdfs['v'].isin(route)]

    # Drop unnecessary columns from the route edges GeoDataFrame
    route_edges_gdf = route_edges_gdf.drop(columns=['oneway', 'lanes', 'bridge', 'reversed', 'geometry'])

    return route_nodes_gdf, route_edges_gdf

In [4]:
def get_min_max_values(route, graph, attribute):
    """
    This function calculates the minimum and maximum values
    of a given attribute (elevation or grade_abs) for nodes or
    edges in the provided route and graph.

    Parameters
    ----------
    route : list
        A list of nodes that make up the route in the graph.
    graph : networkx.MultiDiGraph
        The graph representing the network of nodes and edges.
    attribute : str
        The attribute for which to compute the minimum and maximum values.
        Must be either 'elevation' or 'grade_abs'.

    Returns
    -------
    min_value : float
        The minimum value of the given attribute in the route.
    max_value : float
        The maximum value of the given attribute in the route.
    """
    # Initialize min_value and max_value
    min_value = float('inf')
    max_value = float('-inf')

    # Calculate min_value and max_value for node elevation attribute
    # Calculate min_value and max_value for edge grade_abs attribute
    if attribute == "elevation":
        for node in route:
            value = graph.nodes[node][attribute]
            min_value = min(min_value, value)
            max_value = max(max_value, value)
    elif attribute == "grade_abs":
        orig = route[0]
        for dest in route[1:]:
            edge = graph.edges[(orig, dest, 0)]
            value = edge[attribute]
            min_value = min(min_value, value)
            max_value = max(max_value, value)
            orig = dest

    return min_value, max_value

In [5]:
def get_color_thresholds(min_value, max_value):
    """
    This function calculates two color thresholds
    based on the given minimum and maximum values.
    The thresholds divide the range between min_value
    and max_value into three equal segments.

    Parameters
    ----------
    min_value : float
        The minimum value in the range.
    max_value : float
        The maximum value in the range.

    Returns
    -------
    thresholds : list
        A list containing two threshold values that divide
        the range into three equal segments.
    """
    threshold1 = min_value + (max_value - min_value) / 3
    threshold2 = min_value + (max_value - min_value) * 2 / 3
    return [threshold1, threshold2]

In [6]:
def get_color(value, min_value, max_value):
    """
    Assigns a color ('blue', 'yellow', or 'red') to a value
    based on its position within a specified range.

    Parameters:
    value (float): The input value for which the color will be assigned.
    min_value (float): The lower bound of the range.
    max_value (float): The upper bound of the range.

    Returns:
    str: A string representing the assigned color:
        - 'blue': If the value is in the lower third of the range.
        - 'yellow': If the value is in the middle third of the range.
        - 'red': If the value is in the upper third of the range.
    """
    if value <= min_value + (max_value - min_value) / 3:
        color = 'blue'
    elif value <= min_value + (max_value - min_value) * 2 / 3:
        color = 'yellow'
    else:
        color = 'red'
    return color

In [7]:
# Originally got this code from my coworker; gave it to ChatGPT to make legends not change when layers are toggled.
class BindColormap(MacroElement):
    """Binds a colormap to a given layer.

    Parameters
    ----------
    colormap : branca.colormap.ColorMap
        The colormap to bind.
    """
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
        {% endmacro %}
        """)  # noqa

In [8]:
def create_folium_map_with_colormap_legends(graph, route, grade_filter):
    """
    This function creates a folium map with colormap legends for the given
    graph, route, and grade filter. It plots the route on the map, along
    with node elevations and edge grades, using different colors to represent
    different elevation and grade values. The function also applies a grade
    filter to highlight edges with a grade equal to or greater than the specified
    filter value.

    Parameters
    ----------
    graph : networkx.MultiDiGraph
        The graph representing the network of nodes and edges.
    route : list
        A list of nodes that make up the route to be plotted on the map.
    grade_filter : float
        A threshold value for the edge grade.
        Edges with a grade equal to or greater than this value will be highlighted in red.

    Returns
    -------
    web_map : folium.Map
        The resulting folium map with colormap legends.
    """
    # Create route_nodes_gdf and route_edges_gdf from the graph and route
    route_nodes_gdf, route_edges_gdf = get_route_gdfs(graph, route)

    # Create initial folium map
    web_map = ox.plot_route_folium(graph, route, weight=1)

    # Initialize feature groups for node elevation, edge grade, and edge grade filter
    node_elevation_feature_group = folium.FeatureGroup('Node Elevation', show=False)
    edge_grade_feature_group = folium.FeatureGroup('Edge Grade', show=False)
    edge_grade_filter_feature_group = folium.FeatureGroup(f'Edge Grade Filter on {grade_filter}', show=False)

    # Get min and max values for route node elevations and edge grades
    min_route_node_elev, max_route_node_elev = get_min_max_values(route, graph, 'elevation')
    min_route_edge_grade, max_route_edge_grade = get_min_max_values(route, graph, 'grade_abs')

    # Create colormaps for node elevation, edge grade, and edge grade filter
    elevation_thresholds = get_color_thresholds(min_route_node_elev, max_route_node_elev)
    grade_thresholds = get_color_thresholds(min_route_edge_grade, max_route_edge_grade)
    nodeElevationColormap = cmp.StepColormap(colors=['blue', 'yellow', 'red'],
                                             index=[min_route_node_elev, elevation_thresholds[0], elevation_thresholds[1]],
                                             vmin=min_route_node_elev,
                                             vmax=max_route_node_elev,
                                             caption='Node Elevation')
    edgeGradeColormap = cmp.StepColormap(colors=['blue', 'yellow', 'red'],
                                         index=[min_route_edge_grade, grade_thresholds[0], grade_thresholds[1]],
                                         vmin=min_route_edge_grade,
                                         vmax=max_route_edge_grade,
                                         caption='Edge Grade')
    edgeGradeFilterColormap = cmp.StepColormap(colors=['blue', 'red'],
                                               index=[grade_filter],
                                               vmin=min_route_edge_grade,
                                               vmax=max_route_edge_grade,
                                               caption='Grade of edge')

    # Iterate through each node in the route and create CircleMarkers with the elevation color
    # Add each CircleMarker to the node_elevation_feature_group
    for node in route:
        node_lat = graph.nodes[node]['y']
        node_long = graph.nodes[node]['x']
        node_elev = graph.nodes[node]['elevation']

        # Assign color based on elevation value
        node_color = get_color(node_elev, min_route_node_elev, max_route_node_elev)

        node_popup = \
        f"""Lat: {str(node_lat)}
        Long: {str(node_long)}
        Node: {node}
        Elevation: {node_elev}"""
        folium.CircleMarker((node_lat, node_long),
                            radius=1,
                            color=node_color,
                            popup=node_popup
                            ).add_to(node_elevation_feature_group)

    # Iterate through each edge in the route
    # Get edge grade and location information
    # Assign color based on edge grade
    # Create and add PolyLines to edge_grade_feature_group and
    # edge_grade_filter_feature_group based on edge grade and grade filter
    orig = route[0]
    for dest in route[1:]:
        edge = route_edges_gdf.loc[(route_edges_gdf['u'] == orig) &
                             (route_edges_gdf['v'] == dest)]
        edge_grade = edge['grade_abs'].iloc[0]
        orig_lat = graph.nodes[orig]['y']
        orig_long = graph.nodes[orig]['x']
        dest_lat = graph.nodes[dest]['y']
        dest_long = graph.nodes[dest]['x']

        # Assign color based on edge grade
        edge_color = get_color(edge_grade, min_route_edge_grade, max_route_edge_grade)

        # create color filter
        if edge_grade >= grade_filter:
            edge_color_filter = 'red'
        else:
            edge_color_filter = 'blue'

        edge_locations = [(orig_lat, orig_long),
                        (dest_lat, dest_long)]
        edge_popup = \
        f"""Grade: {edge_grade}"""
        folium.PolyLine(edge_locations,
                        popup=edge_popup,
                        color=edge_color,
                        weight=5).add_to(edge_grade_feature_group)
        folium.PolyLine(edge_locations,
                        popup=edge_popup,
                        color=edge_color_filter,
                        weight=5).add_to(edge_grade_filter_feature_group)
        orig = dest


    # Add feature groups, colormaps, and layer control to the folium map
    nodeElevationColormap.add_to(web_map)
    edgeGradeColormap.add_to(web_map)
    edgeGradeFilterColormap.add_to(web_map)
    node_elevation_feature_group.add_to(web_map)
    edge_grade_feature_group.add_to(web_map)
    edge_grade_filter_feature_group.add_to(web_map)
    folium.LayerControl().add_to(web_map)
    web_map.add_child(
        BindColormap(node_elevation_feature_group, nodeElevationColormap)).add_child(
        BindColormap(edge_grade_feature_group, edgeGradeColormap)).add_child(
        BindColormap(edge_grade_filter_feature_group,edgeGradeFilterColormap)
        )

    # Return the resulting folium map
    return web_map

In [9]:
def find_graph_bbox(coords_list):
    """
    This function takes a list of coordinates and returns
    a bounding box with an additional margin. The bounding
    box is a tuple containing the northern, western, southern,
    and eastern extents of the box.

    Parameters
    ----------
    coords_list : list of tuples
        A list of coordinate tuples (latitude, longitude) for
        which to compute the bounding box.

    Returns
    -------
    bbox : tuple
        A tuple containing the northern, western, southern, and
        eastern extents of the bounding box (with margin).
    """
    # Define the margin to be added to the bounding box
    bbox_margin = 0.05

    # Unzip the list of coordinates into separate latitude and longitude lists
    lats, longs = zip(*coords_list)

    # Determine the northern, southern, eastern, and western coordinates
    northern_lat = max(lats)
    southern_lat = min(lats)
    eastern_long = max(longs)
    western_long = min(longs)

    # Return the bounding box with added margin
    return (northern_lat+bbox_margin,
            western_long-bbox_margin,
            southern_lat-bbox_margin,
            eastern_long+bbox_margin)

In [10]:
def create_graph_route_folium_map(start_address, end_address):
    """
    This function creates a folium map with a route between the
    given start and end addresses. It first geocodes the addresses
    to get their coordinates and creates a bounding box around them.
    It then retrieves the road network within the bounding box and finds
    the shortest path between the start and end nodes. Finally, it plots
    the route on a folium map and returns the graph, route, and map.

    Parameters
    ----------
    start_address : str
        The start address of the route.
    end_address : str
        The end address of the route.

    Returns
    -------
    graph : networkx.MultiDiGraph
        The graph representing the road network within the bounding box.
    route : list
        A list of nodes that make up the shortest route between the start and end addresses.
    route_map : folium.Map
        The folium map with the plotted route.
    """
    # Geocode the start and end addresses to get their coordinates
    start_coord = ox.geocode(start_address)
    finish_coord = ox.geocode(end_address)

    # Define the bounding box of the graph
    coords = [list(start_coord), list(finish_coord)]
    bbox = find_graph_bbox(coords)

    # Get the road network within the bounding box
    graph = ox.graph_from_bbox(north=bbox[0],
                                   west=bbox[1],
                                   south=bbox[2],
                                   east=bbox[3],
                                   network_type='drive',
                                   simplify=False)

    # Find the nearest node to the start and end coordinates
    origin_node = ox.distance.nearest_nodes(graph, start_coord[1], start_coord[0])
    dest_node = ox.distance.nearest_nodes(graph, finish_coord[1], finish_coord[0])

    # Compute the shortest path between the start and end nodes
    route = nx.shortest_path(graph, origin_node, dest_node)

    # Create folium map of the route
    route_map = ox.plot_route_folium(graph, route, weight=1)

    # Return the graph, route, and plotted route
    return graph, route, route_map

In [11]:
# Create a osmnx graph, find the shortest route between two addresses, and plot it on a Folium map
i80_graph, i80_route, i80_route_map = create_graph_route_folium_map('3428 Auburn Blvd, Sacramento, CA 95821',
                                                                    '21784 Donner Pass Rd, Soda Springs, CA 95728')
# i80_route_map

# add elevation data to the route nodes using Google Elevationa API
# add grade data to the route edges using Google Elevation API
i80_graph_elev_data = ox.elevation.add_node_elevations_google(i80_graph.subgraph(i80_route), api_key=google_elevation_api_key)
i80_graph_elev_data = ox.elevation.add_edge_grades(i80_graph_elev_data.subgraph(i80_route))

# create a Folium map with the route and color-coded segments based on elevation and grade
i80_route_final_web_map = create_folium_map_with_colormap_legends(i80_graph_elev_data, i80_route, grade_filter = 0.01)
# i80_route_final_web_map

# output the map to an HTML file
i80_route_final_web_map.save('i80_route_final_web_map.html')